<a href="https://colab.research.google.com/github/Lacave91/lab-sql-query-from-table-names/blob/main/lab-sql-query-from-table-names.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# SQL query from table names

In This notebook we are going to test if using just the name of the table, and a shord definition of its contect we can use a model like GTP3.5-Turbo to select which tables are necessary to create a SQL Order to answer the user petition.

In [6]:
!pip install dotenv

In [7]:
from openai import OpenAI
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())

OPENAI_API_KEY  = os.getenv('OPENAI_API_KEY')

In [8]:
#Functio to call the model.
def return_OAI(user_message):
    client = OpenAI(
    # This is the default and can be omitted
    api_key=OPENAI_API_KEY,
)
    context = []
    context.append({'role':'system', "content": user_message})

    response = client.chat.completions.create(
            model="gpt-3.5-turbo",
            messages=context,
            temperature=0,
        )

    return (response.choices[0].message.content)

In [15]:
# Definition of the tables.
import pandas as pd

# Table and definitions sample
data = {
    'table': ['employees', 'salary', 'studies'],  #
    'definition': [
        'Employee information, name, position, department, etc.',
        'Salary details for each year',
        'Educational studies, name of the institution, degree, etc.'
    ]
}

df = pd.DataFrame(data)
print(df)

       table                                         definition
0  employees  Employee information, name, position, departme...
1     salary                       Salary details for each year
2    studies  Educational studies, name of the institution, ...


In [16]:
text_tables = '\n'.join([f"{row['table']}: {row['definition']}" for index, row in df.iterrows()])

In [17]:
print(text_tables)

employees: Employee information, name, position, department, etc.
salary: Salary details for each year
studies: Educational studies, name of the institution, degree, etc.


In [18]:
prompt_question_tables = """
Given the following tables and their content definitions,
###Tables
{tables}

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
{question}
"""


In [24]:
# Example user question
user_question = "Which employees have the highest salary in the company?"

# Creating the prompt with the user question and table definitions
pqt1 = prompt_question_tables.format(tables=text_tables, question=user_question)

# Printing the final prompt
print(pqt1)



Given the following tables and their content definitions,
###Tables
employees: Employee information, name, position, department, etc.
salary: Salary details for each year
studies: Educational studies, name of the institution, degree, etc.

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
Which employees have the highest salary in the company?



In [25]:
print(return_OAI(pqt1))

{
    "tables": ["employees", "salary"]
}


In [27]:
pqt3 = prompt_question_tables.format(tables=text_tables,
                                     question= "Who has the most studies?")
print(pqt3)


Given the following tables and their content definitions,
###Tables
employees: Employee information, name, position, department, etc.
salary: Salary details for each year
studies: Educational studies, name of the institution, degree, etc.

Tell me which tables would be necessary to query with SQL to address the user's question below.
Return the table names in a json format.
###User Questyion:
Who has the most studies?



In [28]:
print(return_OAI(pqt3))

```json
{
    "tables": ["employees", "studies"]
}
```


# Exercise
 - Complete the prompts similar to what we did in class.
     - Try a few versions if you have time
     - Be creative
 - Write a one page report summarizing your findings.
     - Were there variations that didn't work well? i.e., where GPT either hallucinated or wrong
 - What did you learn?